# Healthcare Data Challenge
- Program Created by Alec Gilfillan
- 8/16/2021

In [1]:
# Import packages
import pandas as pd
from pandasql import sqldf
from datetime import datetime, date

In [2]:
pysqldf = lambda q: sqldf(q, globals())

In [3]:
# Read in Bill_Amount
bill_amount = pd.read_csv('bill_amount.csv')

In [4]:
bill_amount.head(5)

,bill_id,amount
0,40315104,1552.634830
1,2660045161,1032.011951
2,1148334643,6469.605351
3,3818426276,755.965425
4,9833541918,897.347816


In [5]:
bill_amount['amount'].describe()

count    13600.000000
mean      5464.767783
std       7717.479509
min         79.496707
25%        950.689008
50%       1516.951136
75%       7307.056573
max      81849.751340
Name: amount, dtype: float64

In [6]:
pysqldf( """ SELECT  COUNT(*)                    as N_Obs,
                     COUNT(DISTINCT Bill_ID)     as N_Unqiue_Bill_IDs
             FROM bill_amount
         """)

,N_Obs,N_Unqiue_Bill_IDs
0,13600,13600


In [7]:
bill_id = pd.read_csv('bill_id.csv')

In [8]:
bill_id.head(5)

,bill_id,patient_id,date_of_admission
0,7968360812,1d21f2be18683991eb93d182d6b2d220,2011-01-01
1,6180579974,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01
2,7512568183,1d21f2be18683991eb93d182d6b2d220,2011-01-01
3,3762633379,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01
4,7654730355,1d21f2be18683991eb93d182d6b2d220,2011-01-01


In [9]:
pysqldf( """ SELECT  COUNT(*)                    as N_Obs,
                     COUNT(DISTINCT bill_id)     as N_Unique_Bill_IDs,
                     COUNT(DISTINCT patient_id)  as N_Unique_Patient_IDs
             FROM bill_id
         """)

,N_Obs,N_Unique_Bill_IDs,N_Unique_Patient_IDs
0,13600,13600,3000


In [10]:
# Merge bill_id and bill_amount
bills_merged = pysqldf( """ SELECT  A.bill_id,
                                    A.patient_id,
                                    A.date_of_admission,
                                    B.amount
                            FROM bill_id a
                            INNER JOIN bill_amount b
                            ON a.bill_id = b.bill_id
                     """)

In [11]:
bills_merged.head()

,bill_id,patient_id,date_of_admission,amount
0,7968360812,1d21f2be18683991eb93d182d6b2d220,2011-01-01,951.214467
1,6180579974,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01,19900.297086
2,7512568183,1d21f2be18683991eb93d182d6b2d220,2011-01-01,12338.269972
3,3762633379,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01,1145.372193
4,7654730355,1d21f2be18683991eb93d182d6b2d220,2011-01-01,1133.601902


In [12]:
bills_cleaned = pysqldf( """ SELECT  patient_id,
                                     date_of_admission,
                                     SUM(amount) as amount_by_date
                             FROM bills_merged
                             GROUP BY patient_id, date_of_admission
                         """)

In [13]:
bills_cleaned.head()

,patient_id,date_of_admission,amount_by_date
0,00225710a878eff524a1d13be817e8e2,2014-04-10,5190.566695
1,0029d90eb654699c18001c17efb0f129,2012-11-07,22601.497872
2,0040333abd68527ecb53e1db9073f52e,2013-01-19,17447.181635
3,00473b58e3dc8ae37b3cb34069705083,2014-02-10,15285.883220
4,0078662d1d983dde68ea057c42d5b5cf,2012-04-28,73477.869010


In [14]:
pysqldf( """ SELECT  COUNT(*)                    as N_Obs,
                     COUNT(DISTINCT patient_id)  as N_Unique_Patient_IDs
             FROM bills_cleaned
         """)

,N_Obs,N_Unique_Patient_IDs
0,3400,3000


In [15]:
#Check for missing values
bills_cleaned.isnull().sum()

patient_id           0
date_of_admission    0
amount_by_date       0
dtype: int64

In [16]:
bills_cleaned['date_of_admission'] = pd.to_datetime(bills_cleaned['date_of_admission'])

In [17]:
bills_cleaned.dtypes

patient_id                   object
date_of_admission    datetime64[ns]
amount_by_date              float64
dtype: object

In [18]:
bills_cleaned.head(5)

,patient_id,date_of_admission,amount_by_date
0,00225710a878eff524a1d13be817e8e2,2014-04-10,5190.566695
1,0029d90eb654699c18001c17efb0f129,2012-11-07,22601.497872
2,0040333abd68527ecb53e1db9073f52e,2013-01-19,17447.181635
3,00473b58e3dc8ae37b3cb34069705083,2014-02-10,15285.883220
4,0078662d1d983dde68ea057c42d5b5cf,2012-04-28,73477.869010


In [19]:
#Read in clinical data
clinical_data = pd.read_csv('clinical_data.csv')

In [20]:
clinical_data.head(10)

,id,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,...,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height
0,00225710a878eff524a1d13be817e8e2,4/10/14,4/22/14,0,0.0,0,0,0.0,0,1,...,0,0,1,0,0,13.4,27.9,96,66.9,155
1,0029d90eb654699c18001c17efb0f129,11/7/12,11/20/12,0,1.0,0,0,0.0,1,0,...,0,1,0,0,1,16.7,26.5,109,89.1,160
2,0040333abd68527ecb53e1db9073f52e,1/19/13,1/31/13,0,1.0,0,0,0.0,0,1,...,0,1,0,0,1,14.8,25.2,96,79.5,172
3,00473b58e3dc8ae37b3cb34069705083,2/10/14,2/15/14,1,0.0,0,0,0.0,0,0,...,0,1,1,1,0,14.9,28.7,122,81.1,160
4,0078662d1d983dde68ea057c42d5b5cf,4/28/12,5/10/12,0,1.0,No,0,0.0,1,0,...,0,1,0,1,1,14.2,27.2,89,74.7,173
5,0088bbd94c90bbc9158e13465441ebb6,11/5/15,11/15/15,0,1.0,No,0,0.0,0,0,...,1,1,1,0,0,13.0,25.8,102,61.1,160
6,0092d20dd0719e88ecfdd02d5e7ca7b7,1/20/13,1/31/13,0,0.0,No,0,0.0,0,0,...,1,1,1,0,0,14.3,24.7,123,70.9,172
7,01081b8bcb0f9e27b888d23bb35a50dd,2/2/11,2/12/11,0,0.0,0,0,0.0,0,0,...,1,1,1,1,1,13.2,32.2,70,79.3,162
8,010cf28233f50d3857b499a9c208c610,2/3/11,2/11/11,0,0.0,No,0,NaN,1,0,...,1,1,0,0,1,13.6,30.6,102,68.7,165
9,0116c5d9b6c2c2e8758c75e6da86014b,9/1/12,9/10/12,1,0.0,0,0,0.0,0,1,...,1,0,0,1,1,15.1,27.3,112,84.6,154


In [21]:
pysqldf( """ SELECT  COUNT(*)            as N_Obs,
                     COUNT(DISTINCT ID)  as N_Unique_IDs
             FROM clinical_data
         """)

,N_Obs,N_Unique_IDs
0,3400,3000


In [22]:
clinical_data['medical_history_3'].value_counts(dropna=False)

0      2176
No      761
1       348
Yes     115
Name: medical_history_3, dtype: int64

In [23]:
clinical_data.loc[(clinical_data.medical_history_3 == 'Yes'),'medical_history_3']='1'
clinical_data.loc[(clinical_data.medical_history_3 == 'No'),'medical_history_3']='0'

In [24]:
clinical_data['medical_history_3'].value_counts(dropna=False)

0    2937
1     463
Name: medical_history_3, dtype: int64

In [25]:
clinical_data.isnull().sum()

id                      0
date_of_admission       0
date_of_discharge       0
medical_history_1       0
medical_history_2     233
medical_history_3       0
medical_history_4       0
medical_history_5     304
medical_history_6       0
medical_history_7       0
preop_medication_1      0
preop_medication_2      0
preop_medication_3      0
preop_medication_4      0
preop_medication_5      0
preop_medication_6      0
symptom_1               0
symptom_2               0
symptom_3               0
symptom_4               0
symptom_5               0
lab_result_1            0
lab_result_2            0
lab_result_3            0
weight                  0
height                  0
dtype: int64

In [26]:
clinical_data_cleaned = clinical_data.fillna(0)

In [27]:
clinical_data_cleaned.isnull().sum()

id                    0
date_of_admission     0
date_of_discharge     0
medical_history_1     0
medical_history_2     0
medical_history_3     0
medical_history_4     0
medical_history_5     0
medical_history_6     0
medical_history_7     0
preop_medication_1    0
preop_medication_2    0
preop_medication_3    0
preop_medication_4    0
preop_medication_5    0
preop_medication_6    0
symptom_1             0
symptom_2             0
symptom_3             0
symptom_4             0
symptom_5             0
lab_result_1          0
lab_result_2          0
lab_result_3          0
weight                0
height                0
dtype: int64

In [28]:
clinical_data_cleaned.dtypes

id                     object
date_of_admission      object
date_of_discharge      object
medical_history_1       int64
medical_history_2     float64
medical_history_3      object
medical_history_4       int64
medical_history_5     float64
medical_history_6       int64
medical_history_7       int64
preop_medication_1      int64
preop_medication_2      int64
preop_medication_3      int64
preop_medication_4      int64
preop_medication_5      int64
preop_medication_6      int64
symptom_1               int64
symptom_2               int64
symptom_3               int64
symptom_4               int64
symptom_5               int64
lab_result_1          float64
lab_result_2          float64
lab_result_3            int64
weight                float64
height                  int64
dtype: object

In [29]:
clinical_data_cleaned["medical_history_3"] = pd.to_numeric(clinical_data_cleaned["medical_history_3"])

In [30]:
clinical_data_cleaned.dtypes

id                     object
date_of_admission      object
date_of_discharge      object
medical_history_1       int64
medical_history_2     float64
medical_history_3       int64
medical_history_4       int64
medical_history_5     float64
medical_history_6       int64
medical_history_7       int64
preop_medication_1      int64
preop_medication_2      int64
preop_medication_3      int64
preop_medication_4      int64
preop_medication_5      int64
preop_medication_6      int64
symptom_1               int64
symptom_2               int64
symptom_3               int64
symptom_4               int64
symptom_5               int64
lab_result_1          float64
lab_result_2          float64
lab_result_3            int64
weight                float64
height                  int64
dtype: object

In [31]:
clinical_data_cleaned['date_of_admission'] = pd.to_datetime(clinical_data_cleaned['date_of_admission'])
clinical_data_cleaned['date_of_discharge'] = pd.to_datetime(clinical_data_cleaned['date_of_discharge'])

In [32]:
clinical_data_cleaned.head(5)

,id,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,...,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height
0,00225710a878eff524a1d13be817e8e2,2014-04-10,2014-04-22,0,0.0,0,0,0.0,0,1,...,0,0,1,0,0,13.4,27.9,96,66.9,155
1,0029d90eb654699c18001c17efb0f129,2012-11-07,2012-11-20,0,1.0,0,0,0.0,1,0,...,0,1,0,0,1,16.7,26.5,109,89.1,160
2,0040333abd68527ecb53e1db9073f52e,2013-01-19,2013-01-31,0,1.0,0,0,0.0,0,1,...,0,1,0,0,1,14.8,25.2,96,79.5,172
3,00473b58e3dc8ae37b3cb34069705083,2014-02-10,2014-02-15,1,0.0,0,0,0.0,0,0,...,0,1,1,1,0,14.9,28.7,122,81.1,160
4,0078662d1d983dde68ea057c42d5b5cf,2012-04-28,2012-05-10,0,1.0,0,0,0.0,1,0,...,0,1,0,1,1,14.2,27.2,89,74.7,173


In [33]:
clinical_data_cleaned.dtypes

id                            object
date_of_admission     datetime64[ns]
date_of_discharge     datetime64[ns]
medical_history_1              int64
medical_history_2            float64
medical_history_3              int64
medical_history_4              int64
medical_history_5            float64
medical_history_6              int64
medical_history_7              int64
preop_medication_1             int64
preop_medication_2             int64
preop_medication_3             int64
preop_medication_4             int64
preop_medication_5             int64
preop_medication_6             int64
symptom_1                      int64
symptom_2                      int64
symptom_3                      int64
symptom_4                      int64
symptom_5                      int64
lab_result_1                 float64
lab_result_2                 float64
lab_result_3                   int64
weight                       float64
height                         int64
dtype: object

In [34]:
clinical_data_cleaned['length_of_stay'] =  clinical_data_cleaned['date_of_discharge'] - clinical_data_cleaned['date_of_admission']

In [35]:
clinical_data_cleaned.head(5)

,id,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,...,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height,length_of_stay
0,00225710a878eff524a1d13be817e8e2,2014-04-10,2014-04-22,0,0.0,0,0,0.0,0,1,...,0,1,0,0,13.4,27.9,96,66.9,155,12 days
1,0029d90eb654699c18001c17efb0f129,2012-11-07,2012-11-20,0,1.0,0,0,0.0,1,0,...,1,0,0,1,16.7,26.5,109,89.1,160,13 days
2,0040333abd68527ecb53e1db9073f52e,2013-01-19,2013-01-31,0,1.0,0,0,0.0,0,1,...,1,0,0,1,14.8,25.2,96,79.5,172,12 days
3,00473b58e3dc8ae37b3cb34069705083,2014-02-10,2014-02-15,1,0.0,0,0,0.0,0,0,...,1,1,1,0,14.9,28.7,122,81.1,160,5 days
4,0078662d1d983dde68ea057c42d5b5cf,2012-04-28,2012-05-10,0,1.0,0,0,0.0,1,0,...,1,0,1,1,14.2,27.2,89,74.7,173,12 days


In [36]:
demographics = pd.read_csv('demographics.csv')

In [37]:
demographics.head(5)

,patient_id,gender,race,resident_status,date_of_birth
0,fa2d818b2261e44e30628ad1ac9cc72c,Female,Indian,Singaporean,1971-05-14
1,5b6477c5de78d0b138e3b0c18e21d0ae,f,Chinese,Singapore citizen,1976-02-18
2,320aa16c61937447fd6631bf635e7fde,Male,Chinese,Singapore citizen,1982-07-03
3,c7f3881684045e6c49020481020fae36,Male,Malay,Singapore citizen,1947-06-15
4,541ad077cb4a0e64cc422673afe28aef,m,Chinese,Singaporean,1970-12-12


In [38]:
pysqldf( """ SELECT  COUNT(*)                   as N_Obs,
                     COUNT(DISTINCT PATIENT_ID) as N_Unique_IDs
             FROM demographics
         """)

,N_Obs,N_Unique_IDs
0,3000,3000


In [39]:
demographics['gender'].value_counts()

Female    1396
Male      1333
m          170
f          101
Name: gender, dtype: int64

In [40]:
demographics['race'].value_counts()

Chinese    1608
Malay       629
chinese     307
Indian      195
Others      161
India       100
Name: race, dtype: int64

In [41]:
demographics['resident_status'].value_counts()

Singaporean          1782
Singapore citizen     610
PR                    465
Foreigner             143
Name: resident_status, dtype: int64

In [42]:
demographics_cleaned = pysqldf(

 """ SELECT   patient_id,

             CASE
                 WHEN gender IN ('Female', 'f') THEN 1
                 ELSE 0
             END                                    AS female,

             CASE WHEN race in ('Chinese','chinese') THEN 1 ELSE 0 END AS race_chinese,
             CASE WHEN race =   'Malay'              THEN 1 ELSE 0 END AS race_malay,
             CASE WHEN race in ('Indian','India')    THEN 1 ELSE 0 END AS race_indian,
             CASE WHEN race =   'Others'             THEN 1 ELSE 0 END AS race_other,

             CASE WHEN resident_status IN ('Singapore citizen','Singaporean') THEN 1 ELSE 0 END AS resident_singapore,
             CASE WHEN resident_status = 'PR'                                 THEN 1 ELSE 0 END AS resident_pr,
             CASE WHEN resident_status = 'Foreigner'                          THEN 1 ELSE 0 END AS resident_foreign,

             date_of_birth,
             gender,
             race,
             resident_status

    FROM demographics
         """)

In [43]:
demographics_cleaned[['gender','female']].value_counts()

gender  female
Female  1         1396
Male    0         1333
m       0          170
f       1          101
dtype: int64

In [44]:
demographics_cleaned[['race','race_chinese','race_malay','race_indian','race_other']].value_counts()

race     race_chinese  race_malay  race_indian  race_other
Chinese  1             0           0            0             1608
Malay    0             1           0            0              629
chinese  1             0           0            0              307
Indian   0             0           1            0              195
Others   0             0           0            1              161
India    0             0           1            0              100
dtype: int64

In [45]:
demographics_cleaned[['resident_status','resident_singapore','resident_pr','resident_foreign']].value_counts()

resident_status    resident_singapore  resident_pr  resident_foreign
Singaporean        1                   0            0                   1782
Singapore citizen  1                   0            0                    610
PR                 0                   1            0                    465
Foreigner          0                   0            1                    143
dtype: int64

In [46]:
demographics_cleaned2 = demographics_cleaned.drop(columns=['gender', 'race','resident_status'])

In [47]:
demographics_cleaned2.head(5)

,patient_id,female,race_chinese,race_malay,race_indian,race_other,resident_singapore,resident_pr,resident_foreign,date_of_birth
0,fa2d818b2261e44e30628ad1ac9cc72c,1,0,0,1,0,1,0,0,1971-05-14
1,5b6477c5de78d0b138e3b0c18e21d0ae,1,1,0,0,0,1,0,0,1976-02-18
2,320aa16c61937447fd6631bf635e7fde,0,1,0,0,0,1,0,0,1982-07-03
3,c7f3881684045e6c49020481020fae36,0,0,1,0,0,1,0,0,1947-06-15
4,541ad077cb4a0e64cc422673afe28aef,0,1,0,0,0,1,0,0,1970-12-12


In [48]:
def age(born):
    born = datetime.strptime(born, "%Y-%m-%d").date()
    today = date.today()
    return today.year - born.year - ((today.month, 
                                      today.day) < (born.month, 
                                                    born.day))

In [49]:
demographics_cleaned2['age'] = demographics_cleaned2['date_of_birth'].apply(age)

In [50]:
demographics_cleaned2.head(5)

,patient_id,female,race_chinese,race_malay,race_indian,race_other,resident_singapore,resident_pr,resident_foreign,date_of_birth,age
0,fa2d818b2261e44e30628ad1ac9cc72c,1,0,0,1,0,1,0,0,1971-05-14,50
1,5b6477c5de78d0b138e3b0c18e21d0ae,1,1,0,0,0,1,0,0,1976-02-18,45
2,320aa16c61937447fd6631bf635e7fde,0,1,0,0,0,1,0,0,1982-07-03,39
3,c7f3881684045e6c49020481020fae36,0,0,1,0,0,1,0,0,1947-06-15,74
4,541ad077cb4a0e64cc422673afe28aef,0,1,0,0,0,1,0,0,1970-12-12,50


In [51]:
clinical_data_merged = pysqldf( """ SELECT   A.*,
                                             B.female,
                                             B.race_chinese,
                                             B.race_malay,
                                             B.race_indian,
                                             B.race_other,
                                             B.resident_singapore,
                                             B.resident_pr,
                                             B.resident_foreign,
                                             B.date_of_birth,
                                             B.age,
                                             C.amount_by_date
                                             
                                    FROM clinical_data_cleaned A
                                    LEFT JOIN demographics_cleaned2 B
                                    ON A.id = B.patient_id
                                    LEFT JOIN bills_cleaned C
                                    ON A.id = C.patient_id and A.date_of_admission = C.date_of_admission
                                """)

/opt/anaconda3/lib/python3.8/site-packages/pandasql/sqldf.py:120: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  to_sql(df, name=tablename, con=conn,


In [52]:
clinical_data_merged.head(10)

,id,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,...,race_chinese,race_malay,race_indian,race_other,resident_singapore,resident_pr,resident_foreign,date_of_birth,age,amount_by_date
0,00225710a878eff524a1d13be817e8e2,2014-04-10 00:00:00.000000,2014-04-22 00:00:00.000000,0,0.0,0,0,0.0,0,1,...,1,0,0,0,1,0,0,1983-01-16,38,5190.566695
1,0029d90eb654699c18001c17efb0f129,2012-11-07 00:00:00.000000,2012-11-20 00:00:00.000000,0,1.0,0,0,0.0,1,0,...,1,0,0,0,1,0,0,1943-10-14,77,22601.497872
2,0040333abd68527ecb53e1db9073f52e,2013-01-19 00:00:00.000000,2013-01-31 00:00:00.000000,0,1.0,0,0,0.0,0,1,...,0,0,1,0,1,0,0,1972-08-26,48,17447.181635
3,00473b58e3dc8ae37b3cb34069705083,2014-02-10 00:00:00.000000,2014-02-15 00:00:00.000000,1,0.0,0,0,0.0,0,0,...,1,0,0,0,1,0,0,1976-07-23,45,15285.883220
4,0078662d1d983dde68ea057c42d5b5cf,2012-04-28 00:00:00.000000,2012-05-10 00:00:00.000000,0,1.0,0,0,0.0,1,0,...,0,1,0,0,0,0,1,1942-10-19,78,73477.869010
5,0088bbd94c90bbc9158e13465441ebb6,2015-11-05 00:00:00.000000,2015-11-15 00:00:00.000000,0,1.0,0,0,0.0,0,0,...,0,0,0,1,0,0,1,1960-10-22,60,26051.592258
6,0092d20dd0719e88ecfdd02d5e7ca7b7,2013-01-20 00:00:00.000000,2013-01-31 00:00:00.000000,0,0.0,0,0,0.0,0,0,...,0,1,0,0,1,0,0,1947-09-18,73,21498.986758
7,01081b8bcb0f9e27b888d23bb35a50dd,2011-02-02 00:00:00.000000,2011-02-12 00:00:00.000000,0,0.0,0,0,0.0,0,0,...,1,0,0,0,1,0,0,1948-04-12,73,27512.273964
8,010cf28233f50d3857b499a9c208c610,2011-02-03 00:00:00.000000,2011-02-11 00:00:00.000000,0,0.0,0,0,0.0,1,0,...,0,1,0,0,1,0,0,1973-05-28,48,25612.364148
9,0116c5d9b6c2c2e8758c75e6da86014b,2012-09-01 00:00:00.000000,2012-09-10 00:00:00.000000,1,0.0,0,0,0.0,0,1,...,1,0,0,0,1,0,0,1990-12-15,30,15790.597745


In [53]:
pysqldf( """ SELECT  COUNT(*)                    as N_Obs,
                     COUNT(DISTINCT id)          as N_Unique_IDs
             FROM clinical_data_merged
         """)

,N_Obs,N_Unique_IDs
0,3400,3000


In [54]:
clinical_data_merged = clinical_data_merged.rename(columns={"id": "patient_id"})

In [55]:
pysqldf ( """ select *
              from clinical_data_merged
              where patient_id = '012c5eb5397a72f8aeb64f942b60846d'

""")

,patient_id,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,...,race_chinese,race_malay,race_indian,race_other,resident_singapore,resident_pr,resident_foreign,date_of_birth,age,amount_by_date
0,012c5eb5397a72f8aeb64f942b60846d,2011-06-08 00:00:00.000000,2011-06-23 00:00:00.000000,0,0.0,0,0,0.0,1,0,...,0,1,0,0,1,0,0,1979-03-24,42,21479.941481
1,012c5eb5397a72f8aeb64f942b60846d,2015-11-17 00:00:00.000000,2015-11-27 00:00:00.000000,0,0.0,0,0,0.0,1,0,...,0,1,0,0,1,0,0,1979-03-24,42,30485.659969


In [56]:
df = pysqldf( """   SELECT  *, 
                            count(patient_id)   as total_admit,
                            sum(amount_by_date) as total_amount
                    FROM clinical_data_merged
                    GROUP BY patient_id
              """)

In [57]:
df['total_admit'].value_counts(dropna=False)

1    2621
2     359
3      19
4       1
Name: total_admit, dtype: int64

In [58]:
df['total_amount'].describe()

count      3000.000000
mean      24773.613948
std       13846.477234
min        2946.100354
25%       15612.050486
50%       21815.197138
75%       29985.749788
max      167641.308465
Name: total_amount, dtype: float64

In [59]:
df.to_csv('cleaned_df.csv', index=False)

In [60]:
df.head(10)
  

,patient_id,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,...,race_indian,race_other,resident_singapore,resident_pr,resident_foreign,date_of_birth,age,amount_by_date,total_admit,total_amount
0,00225710a878eff524a1d13be817e8e2,2014-04-10 00:00:00.000000,2014-04-22 00:00:00.000000,0,0.0,0,0,0.0,0,1,...,0,0,1,0,0,1983-01-16,38,5190.566695,1,5190.566695
1,0029d90eb654699c18001c17efb0f129,2012-11-07 00:00:00.000000,2012-11-20 00:00:00.000000,0,1.0,0,0,0.0,1,0,...,0,0,1,0,0,1943-10-14,77,22601.497872,1,22601.497872
2,0040333abd68527ecb53e1db9073f52e,2013-01-19 00:00:00.000000,2013-01-31 00:00:00.000000,0,1.0,0,0,0.0,0,1,...,1,0,1,0,0,1972-08-26,48,17447.181635,1,17447.181635
3,00473b58e3dc8ae37b3cb34069705083,2014-02-10 00:00:00.000000,2014-02-15 00:00:00.000000,1,0.0,0,0,0.0,0,0,...,0,0,1,0,0,1976-07-23,45,15285.883220,1,15285.883220
4,0078662d1d983dde68ea057c42d5b5cf,2012-04-28 00:00:00.000000,2012-05-10 00:00:00.000000,0,1.0,0,0,0.0,1,0,...,0,0,0,0,1,1942-10-19,78,73477.869010,1,73477.869010
5,0088bbd94c90bbc9158e13465441ebb6,2015-11-05 00:00:00.000000,2015-11-15 00:00:00.000000,0,1.0,0,0,0.0,0,0,...,0,1,0,0,1,1960-10-22,60,26051.592258,1,26051.592258
6,0092d20dd0719e88ecfdd02d5e7ca7b7,2013-01-20 00:00:00.000000,2013-01-31 00:00:00.000000,0,0.0,0,0,0.0,0,0,...,0,0,1,0,0,1947-09-18,73,21498.986758,1,21498.986758
7,01081b8bcb0f9e27b888d23bb35a50dd,2011-02-02 00:00:00.000000,2011-02-12 00:00:00.000000,0,0.0,0,0,0.0,0,0,...,0,0,1,0,0,1948-04-12,73,27512.273964,1,27512.273964
8,010cf28233f50d3857b499a9c208c610,2011-02-03 00:00:00.000000,2011-02-11 00:00:00.000000,0,0.0,0,0,0.0,1,0,...,0,0,1,0,0,1973-05-28,48,25612.364148,1,25612.364148
9,0116c5d9b6c2c2e8758c75e6da86014b,2012-09-01 00:00:00.000000,2012-09-10 00:00:00.000000,1,0.0,0,0,0.0,0,1,...,0,0,1,0,0,1990-12-15,30,15790.597745,1,15790.597745
